# init

> Router assembly for Phase 2 segmentation routes

In [ ]:
#| default_exp routes.init

In [ ]:
#| export
from typing import List, Dict, Callable, Tuple

from fasthtml.common import APIRouter

from cjm_fasthtml_card_stack.core.models import CardStackUrls

from cjm_transcript_segmentation.models import SegmentationUrls
from cjm_transcript_segmentation.routes.core import (
    WorkflowStateStore, DEFAULT_MAX_HISTORY_DEPTH,
)
from cjm_transcript_segmentation.routes.card_stack import init_card_stack_router
from cjm_transcript_segmentation.routes.handlers import init_workflow_router
from cjm_transcript_segmentation.services.segmentation import SegmentationService
from cjm_transcript_source_select.services.source import SourceService


# Type alias for wrapped handlers dict
WrappedHandlers = Dict[str, Callable]

## Router Assembly

In [ ]:
#| export
def init_segmentation_routers(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    source_service:SourceService,  # Service for fetching source blocks
    segmentation_service:SegmentationService,  # Service for NLTK sentence splitting
    prefix:str,  # Base prefix for segmentation routes (e.g., "/workflow/seg")
    max_history_depth:int=DEFAULT_MAX_HISTORY_DEPTH,  # Maximum history stack depth
    wrapped_handlers:WrappedHandlers=None,  # Dict with 'init', 'split', 'merge', 'undo', 'reset', 'ai_split' keys
) -> Tuple[List[APIRouter], SegmentationUrls, Dict[str, Callable]]:  # (routers, urls, merged_routes)
    """Initialize and return all segmentation routers with URL bundle.
    
    The wrapped_handlers dict should contain handlers that already have
    cross-domain concerns (KB system, alignment status) handled by the
    combined layer's wrapper factories.
    """
    # Create empty URL bundle (populated after routes are defined)
    urls = SegmentationUrls()
    wrapped_handlers = wrapped_handlers or {}

    # Initialize focused routers
    card_stack_router, card_stack_routes = init_card_stack_router(
        state_store, workflow_id, f"{prefix}/card_stack", urls
    )
    workflow_router, workflow_routes = init_workflow_router(
        state_store, workflow_id,
        source_service, segmentation_service,
        f"{prefix}/workflow", urls,
        max_history_depth=max_history_depth,
        handler_init=wrapped_handlers.get("init"),
        handler_split=wrapped_handlers.get("split"),
        handler_merge=wrapped_handlers.get("merge"),
        handler_undo=wrapped_handlers.get("undo"),
        handler_reset=wrapped_handlers.get("reset"),
        handler_ai_split=wrapped_handlers.get("ai_split"),
    )

    # Populate the URL bundle using .to() on route functions
    urls.card_stack = CardStackUrls(
        nav_up=card_stack_routes["nav_up"].to(),
        nav_down=card_stack_routes["nav_down"].to(),
        nav_first=card_stack_routes["nav_first"].to(),
        nav_last=card_stack_routes["nav_last"].to(),
        nav_page_up=card_stack_routes["nav_page_up"].to(),
        nav_page_down=card_stack_routes["nav_page_down"].to(),
        update_viewport=card_stack_routes["update_viewport"].to(),
        save_width=card_stack_routes["save_width"].to(),
    )
    urls.enter_split = card_stack_routes["enter_split"].to()
    urls.exit_split = card_stack_routes["exit_split"].to()
    urls.init = workflow_routes["init"].to()
    urls.split = workflow_routes["split"].to()
    urls.merge = workflow_routes["merge"].to()
    urls.undo = workflow_routes["undo"].to()
    urls.reset = workflow_routes["reset"].to()
    urls.ai_split = workflow_routes["ai_split"].to()

    # Merge all routes for external access
    merged_routes = {
        **card_stack_routes,
        **workflow_routes,
    }

    routers = [card_stack_router, workflow_router]

    return routers, urls, merged_routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()